In [1]:
#import the necessary modules 
%matplotlib inline 
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
#import scipy
import sklearn
import itertools as it
from itertools import cycle 
import os.path as op
import timeit 
import json
from matplotlib import animation
import matplotlib.font_manager as font_manager
from collections import namedtuple
#from functools import partial
#from pathlib import Path

In [2]:
# Set plotting style
plt.style.use('seaborn-white')

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
#import matplotlib.pyplot as plt

In [3]:
%matplotlib widget

In [4]:
import multiprocessing as m_proc
m_proc.cpu_count()

4

### Now use MD Analysis to calculate no. of frames a center PLGA residues and terminal PLGA residue is with 4 Angstroms of BSA (1.2 nm restrained system)

Import MDAnalysis

In [5]:
from prot_polymer_analysis import get_protresd_list, aa_frmcount, grptwocnt_aa, gtwo_trjcnt 
from prot_polymer_analysis import frac_cont, bavg_frac_cnt, prot_poly_cntmovie, AA_list_org

In [6]:
# Import MDAnalysis
import MDAnalysis as mda
import MDAnalysis.analysis.distances as maa_dist

### First table will be total fractional contacts and oligomer occupancy values for each Rg value 

#### Distance-based analysis 

Find residues that have at least one atom within a cutoff $d = 4.0$ Angstrom near water molecules in BSA/water simulation

Calculate the number of surface bsa residues from a 1 ns BSA/water simulation

In [7]:
#Units of Angstroms 
dmax = 4.0 

In [8]:
def middle_of_band(band_start, band_stop, plot_min=0, plot_max=60):
    half_way = (band_stop - band_start) / 2
    mid_band = band_start + half_way
    plot_fraction = (mid_band - plot_min) / (plot_max - plot_min)

    return plot_fraction

# 1.2 nm PLGA restrained Rg 100 ns trajectory

Load the rg = 1.2 nm (3 PLGA N = 20 oligomer/BSA system) 

In [9]:
# Set up the MD Simulation, Make sure you do gmx trjconv -s topol.tpr -f confout.gro  -o new_conf.pdb -dump 0 -n bsaplga_nk.ndx to generate 
# a new pdb file that contains unique chain identifiers 
u_pn20 = mda.Universe("../1.2nm_bsa_prod/k4000_plumed/new_conf.pdb", "../1.2nm_bsa_prod/k4000_plumed/pp_1_2nmres.xtc")

In [10]:
u_pn20

<Universe with 10148 atoms>

Check that we are on the first frame

In [11]:
u_pn20.trajectory.frame

0

In [12]:
pn20_len = len(u_pn20.trajectory)
pn20_len

10001

In [13]:
# Select one polymer chain, heavy atoms only 
#all_pn20 = u_pn20.select_atoms("(resname sPLG PLG tPLG and segid B) and not type H")

#Select all the PLGA residues, heavy atoms only
all_pn20 = u_pn20.select_atoms("resname sPLG PLG tPLG and not type H")
all_pn20

<AtomGroup with 543 atoms>

In [14]:
# Select BSA residues, heavy atoms only 
prot = u_pn20.select_atoms("protein and not type H")
prot

<AtomGroup with 4653 atoms>

Calculate AA frame counts for PLGA residues, 1.2 nm RG restraint, 100ns trajectory 

In [15]:
#dmax = 4.0, protein group(4653 atoms), plga atom group (543 atoms), took 381.6 s (6 min 36s on 4 cores)
start = 0
end = pn20_len - 1
s_time = timeit.default_timer()
h2di = aa_frmcount(prot, all_pn20, dmax, u_pn20, start, end)
timeit.default_timer() - s_time

611.0223658857867

In [ ]:
#h2di

In [16]:
pr_res_n = list(prot.residues)
ss_res_n = [str(row) for row in pr_res_n]
rkg_n = {key:h2di[key][1] for key, value in h2di.items()}
plg_1_2nm_occ = pd.DataFrame(data=ss_res_n, columns=["BSA_des_res"])
plg_1_2nm_occ['mda_occ_1.2nm'] = plg_1_2nm_occ['BSA_des_res'].map(rkg_n)
plg_1_2nm_occ['mda_occ_1.2nm'] = plg_1_2nm_occ['mda_occ_1.2nm'].replace('nan', np.nan).fillna(0)
plg_1_2nm_occ['mda_occ_1.2nm'] = plg_1_2nm_occ['mda_occ_1.2nm'].round(2)
plg_1_2nm_occ

,BSA_des_res,mda_occ_1.2nm
0,"<Residue ASP, 1>",0.00
1,"<Residue THR, 2>",0.00
2,"<Residue HIS, 3>",0.00
3,"<Residue LYS, 4>",0.00
4,"<Residue SER, 5>",0.00
...,...,...
578,"<Residue GLN, 579>",0.00
579,"<Residue THR, 580>",0.00
580,"<Residue ALA, 581>",0.01
581,"<Residue LEU, 582>",0.00


In [ ]:
plg_1_2nm_occ['mda_occ_1.2nm'][plg_1_2nm_occ['mda_occ_1.2nm'] != 0]

In [17]:
bsa_r = np.array(list(prot.resids)) # shape is 4652
m_occ_12r = np.array(list(plg_1_2nm_occ['mda_occ_1.2nm'])) # shape is 583
m_occ = np.zeros(shape=(4653))
at_ind = np.where(bsa_r[:-1] != bsa_r[1:])[0]
at_in_nw = np.sort(np.append([0,4653],at_ind))
nw_v = 0
for i in range(583):
    b = at_in_nw[i+1] +1
    m_occ[nw_v:b] = m_occ_12r[i]
    nw_v = at_in_nw[i+1] + 1 

In [18]:
m_occ[633:646]

array([0.  , 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08,
       0.08, 0.06])

### Visualize Occupanct on protein 

In [ ]:
prot.occupancies = m_occ
prot.occupancies

In [ ]:
with mda.Writer("prot_12.pdb") as pdb:
    pdb.write(prot)

In [ ]:
# Frame count and occupancy for each residue
#h2di

In [ ]:
len(h2di.keys())

### Residue Importance: 1.2 nm restrained 

In [ ]:
# Need to fix function, the residue number are not counting the other 2 PLGA oligomers cuz of same resid number
trj_ppmap_12nm = prot_poly_cntmovie(prot, all_pn20, dmax, u_pn20, 0, 10000)
#trj_ppmap_12nm_chC = prot_poly_cntmovie(prot, all_pn20_C, dmax, u_pn20, 0, 10000)

In [ ]:
np.save('1.2nm_res.npy', trj_ppmap_12nm)    # .npy extension is added if not given

In [19]:
trj_ppmap_12nm = np.load("1.2nm_res.npy", allow_pickle=True)

In [20]:
trj_ppmap_12nm[0].shape

(583, 60)

In [21]:
np.sum(trj_ppmap_12nm[1000][0])

0.0

In [22]:
kj = np.zeros(shape=(10000, 583))
kj[:,582].shape

(10000,)

In [23]:
pp_12nm_ct = np.zeros(shape=(10000, 583))
for i in range(10000):
    for j in range(583):
        pp_12nm_ct[i][j] = np.sum(trj_ppmap_12nm[i][j])

In [24]:
pp_12nmtot = np.zeros(shape=(583))
for i in range(583):
    pp_12nmtot[i] = np.sum(pp_12nm_ct[:,i])
#pp_12nmtot

In [25]:
np.nonzero(pp_12nmtot)

(array([ 40,  41,  44,  59,  71,  72,  74,  75,  76,  77,  78,  79,  80,
         81,  84,  85,  87,  88,  90,  91,  92,  93,  94,  96, 165, 166,
        168, 175, 179, 304, 305, 308, 309, 310, 311, 361, 362, 363, 365,
        366, 367, 369, 370, 372, 373, 374, 376, 377, 378, 380, 381, 383,
        384, 385, 387, 388, 389, 391, 392, 393, 394, 395, 396, 397, 398,
        399, 400, 402, 403, 404, 408, 411, 412, 414, 426, 433, 436, 437,
        438, 440, 443, 470, 484, 486, 487, 488, 489, 490, 491, 492, 493,
        495, 516, 517, 519, 520, 523, 524, 535, 536, 537, 538, 539, 540,
        541, 543, 544, 546, 547, 548, 550, 551, 554, 555, 557, 558, 579,
        580, 582]),)

In [121]:
y_pos = np.arange(583) + 1
wid = np.zeros(shape=583)
wid += 2
#wid
fig = plt.figure(figsize=(12,12))
fig.canvas.layout.width = '800px'
fig.canvas.layout.height = '700px'
plt.bar(y_pos, pp_12nmtot, align='center',width=wid, alpha=1, label='1.2 nm PLGA')
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.xlim([0,600])
plt.ylim([0,16000])
plt.legend(fontsize=14)
plt.ylabel(r'Total No. of PLGA contacts', fontsize=15)
plt.xlabel(r'BSA Residue ID', fontsize=15)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'BSA Residue ID')

### Total number of residues that are within 4 angstroms of a PLGA oligomer residue within a 100 ns trajectory block

In [27]:
a_a = ["GLY","ALA","VAL","LEU","ILE","MET","PHE","TRP","PRO","SER","THR","CYS","TYR","ASN","GLN","ASP"
               ,"GLU","LYS","ARG","HIS"]

In [28]:
# This code chunk gets the BSA residues and their corresponding number in a pandas dataframe 
red_bsa = []
bh = np.arange(0,584)
for i in range(583):
    b_str = str(list(prot.residues[i:i+1]))
    if str(bh[i+1]) in b_str: 
        red_bsa.append(str(b_str[10:13])+" "+str(bh[i+1]))

In [29]:
pr_res = list(prot.residues)
ss_res = [str(row) for row in pr_res]
rkg = {key:h2di[key][0] for key, value in h2di.items()}
plg_1_2nmaa = pd.DataFrame(data=ss_res, columns=["BSA_des_res"])
plg_1_2nmaa['mda_plga_frm_1.2nm'] = plg_1_2nmaa['BSA_des_res'].map(rkg)
plg_1_2nmaa['BSA_des_res'] = red_bsa
plg_1_2nmaa['mda_plga_frm_1.2nm'] = plg_1_2nmaa['mda_plga_frm_1.2nm'].replace('nan', np.nan).fillna(0)
plg_1_2nmaa.tail()

,BSA_des_res,mda_plga_frm_1.2nm
578,GLN 579,0.0
579,THR 580,2.0
580,ALA 581,51.0
581,LEU 582,0.0
582,ALA 583,3.0


In [30]:
# Read in data from the oputput of wrapper.sh, where the frame count is given for each BSA residue that was within 
# 4 angstroms of PLGA trimer 
wat_data = pd.read_csv('occ_BSA1ns.txt', sep=" ", header=None, usecols=None ,index_col=None)
wat_data.columns = ["BSA_res_no","No. of frames (VMD)"]
wat_data = wat_data.drop("BSA_res_no", axis=1)

pr_res = list(prot.residues)
ss_res = [str(row) for row in pr_res]

wat_data['BSA_des_res'] = ss_res
wat_data = wat_data[['BSA_des_res',"No. of frames (VMD)"]]
#wat_data.head()

# load MDAnalysis values from MDA_BSA1ns.txt file(129003 atoms SOL group was used to calc. frame counts for txt.
# file)
h2ob_dict = json.load(open("MDA_BSA1ns.txt"))
wat_data['Mda_frames'] = wat_data['BSA_des_res'].map(h2ob_dict)


# From MD Analysis
#Get the count of bsa residues that have 1001 or 1002 frames ( I ran a 1 ns NPT simulation of 1 BSA in water )

#aa_count = pd.DataFrame(data=a_a)
c_list = []

for i in range(len(a_a)):
    count = 0
    for index, row in wat_data.iterrows():
        if a_a[i] in row["BSA_des_res"]:
            if row['Mda_frames'] == 1001: 
                count += 1
                #c_list.append(str(str(a_a[i])+"  "+str(row['No. of frames']))) 
            elif row['Mda_frames'] == 1000:
                count += 1
                #c_list.append(str(str(a_a[i])+"  "+str(row['No. of frames'])))
    c_list.append(str(str(a_a[i])+"  "+str(count)))
    
#c_list

# From VMD
#Get the count of bsa residues that have 1001 or 1002 frames ( I ran a 1 ns NPT simulation of 1 BSA in water )

#aa_count = pd.DataFrame(data=a_a)
vmd_list = []

for i in range(len(a_a)):
    count = 0
    for index, row in wat_data.iterrows():
        if a_a[i] in row["BSA_des_res"]:
            if row["No. of frames (VMD)"] == 1001: 
                count += 1
                #c_list.append(str(str(a_a[i])+"  "+str(row['No. of frames']))) 
            elif row["No. of frames (VMD)"] == 1002:
                count += 1
                #c_list.append(str(str(a_a[i])+"  "+str(row['No. of frames'])))
    vmd_list.append(str(str(a_a[i])+"  "+str(count)))

# Main difference is that Alanine 583 is counted for all 1001 frames. It seems VMD is unable to calc dist for that res
#vmd_list

#hydrophobic_res = ['ALA', 'ILE', 'LEU', 'VAL', 'GLY', 'PRO','PHE', 'TRP','MET']
#polar_res = ['ASN', 'CYS', 'GLN', 'SER', 'THR','TYR']
#neg_res = ['ASP', 'GLU']
#pos_res = ['ARG', 'HIS', 'LYS']
# aromatic_res = ['PHE', 'TRP', 'TYR', 'HIS']
#all_res = [pos_res, neg_res, polar_res, hydrophobic_res]

# Put the AA count in a pandas dataframe 
dg , ji = AA_list_org(c_list)
aa_count = pd.DataFrame(data=dg, index=None, columns=['Amino_acids'])
new_lf = pd.Series(data=ji, index=None)
vmg, vmdj = AA_list_org(vmd_list)
n2lf = pd.Series(data=vmdj, index=None)
aa_count['No_of_surf_res (MDAnalysis)'] = new_lf
aa_count['No_of_surf_res (VMD)'] = n2lf

In [31]:
apl_list = []

# Some residues don't have any contact with the 3 N = 20 PLGA oligomers within 100 ns,
# Put residues that do have contact with BSA in a separate list
for index, r_pl in plg_1_2nmaa.iterrows():
    if r_pl['mda_plga_frm_1.2nm'] != 0:
        apl_list.append(r_pl['BSA_des_res'])
        
# This chunk of code gets an AA count from the above list, in order 
# to get a total number of residues that contact BSA
cpl_l = []

for index, r_a in aa_count.iterrows():
    count = 0
    for i in range(len(apl_list)):
        if r_a['Amino_acids'] in apl_list[i]:
            count += 1
    cpl_l.append(count)      
        
aa_count['plga_1.2nm_100ns'] = cpl_l
aa_count

,Amino_acids,No_of_surf_res (MDAnalysis),No_of_surf_res (VMD),plga_1.2nm_100ns
0,LYS,59,59,18
1,ARG,23,23,5
2,HIS,16,16,2
3,ASP,40,40,12
4,GLU,59,59,16
5,SER,23,23,3
6,THR,29,29,9
7,CYS,24,24,7
8,TYR,19,19,2
9,ASN,14,14,4


In [32]:
# This gives the total number of residues that are within 4 angstroms of a PLGA oligomer residue
# within a 100 ns trajectory block
aa_count['plga_1.2nm_100ns'].sum()

120

In [33]:
# This gives the total number of residues that are within 4 angstroms of a water molecule
# within a 1 ns trajectory block
aa_count['No_of_surf_res (MDAnalysis)'].sum()

487

In [34]:
# This gives the total fraction of contacts within the 1.2 nm Rg 100 ns trajectory
aa_count['plga_1.2nm_100ns'].sum()/aa_count['No_of_surf_res (MDAnalysis)'].sum()

0.2464065708418891

Calculate mean occupancy and the standard deviation for 1.2 nm trajectory

Numpy mean and std function was used to calculate mean occupancy and std dev using occ values from aa_frmcount output

In [35]:
# Mean occupancy and std deviation 
ll_mo = [value[1] for key, value in h2di.items()]
print("Mean Occpancy (1.2 nm Rg): "+str(np.mean(ll_mo)), "Occ. std. dev.: "+str(np.std(ll_mo)))

Mean Occpancy (1.2 nm Rg): 0.0816425 Occ. std. dev.: 0.12386145799676077


### Calc. fractional contacts for each AA group type 

In [36]:
cd_12nm = frac_cont(h2di)
cd_12nm

{'Negative': [26653.0, 28, 114227.14285714286, 0.2274227983368448],
 'Positive': [25196.0, 25, 120940.79999999999, 0.24078948734185865],
 'Polar': [19656.0, 34, 69374.11764705883, 0.1381217771259086],
 'Hydrophobic': [26466.0, 33, 96239.99999999999, 0.19161093908573842],
 'Aromatic': [5920.0, 7, 101485.71428571429, 0.20205499810964941]}

In [37]:
cd = frac_cont(h2di)
kklh = []
for key, value in cd.items():
    kklh.append(value[1])
# Must substract aromatic residues, since they are already counted
sum(kklh) - cd['Aromatic'][1]

120

In [ ]:
start = 0
end = pn20_len - 1
aa_12nm, l_f12nm = gtwo_trjcnt(prot, all_pn20, dmax, u_pn20, start, end)

In [ ]:
aa_12nm

In [ ]:
l_f12nm.shape

In [38]:
no_surf = aa_count['No_of_surf_res (MDAnalysis)'].sum()
no_surf

487

In [39]:
fcnt_rg1_2nm, prgrp_1_2nm, aa_matx = bavg_frac_cnt(5, prot, all_pn20, dmax, u_pn20, no_surf, 0, 10000)

0
2000
2000
4000
4000
6000
6000
8000
8000
10000


In [40]:
fcnt_rg1_2nm

{'Negative': array([0.29736931, 0.21006099, 0.22729602, 0.2225961 , 0.25079287]),
 'Positive': array([0.35389445, 0.19092597, 0.22382273, 0.24456788, 0.22958072]),
 'Polar': array([0.15894534, 0.20406011, 0.1972663 , 0.09772299, 0.1090522 ]),
 'Hydrophobic': array([0.1816259 , 0.06632579, 0.19620205, 0.23178616, 0.22002748]),
 'Aromatic': array([0.008165  , 0.32862714, 0.1554129 , 0.20332686, 0.19054674]),
 'total_frac': array([0.07186858, 0.14579055, 0.07392197, 0.1026694 , 0.10061602])}

In [41]:
prgrp_1_2nm

{'Negative': array([array([1.456     , 2.06205335]), array([4.857     , 2.54981391]),
        array([6.923     , 3.07133049]), array([6.236     , 3.25780662]),
        array([9.257     , 3.20685999])], dtype=object),
 'Positive': array([array([1.834     , 2.02840923]), array([4.9885    , 3.00189403]),
        array([8.334     , 3.09232016]), array([8.687     , 3.16165004]),
        array([10.92      ,  2.63374258])], dtype=object),
 'Polar': array([array([0.8245    , 1.64550896]), array([4.503     , 3.86768548]),
        array([6.3695    , 3.24314196]), array([4.068     , 2.78089482]),
        array([6.563     , 3.36214678])], dtype=object),
 'Hydrophobic': array([array([0.671     , 1.12194429]), array([1.219    , 2.3375284]),
        array([5.189     , 2.70929493]), array([7.437     , 3.89101928]),
        array([13.6685    ,  3.95393573])], dtype=object),
 'Aromatic': array([array([0.0025    , 0.04993746]), array([2.034     , 1.83080419]),
        array([2.1045    , 2.09178865]), arr

In [42]:
fc_12nm_mean = np.array([np.mean(fcnt_rg1_2nm['Negative']), np.mean(fcnt_rg1_2nm['Positive'])
                        ,np.mean(fcnt_rg1_2nm['Polar']),np.mean(fcnt_rg1_2nm['Hydrophobic'])
                        , np.mean(fcnt_rg1_2nm['Aromatic'])])
fc_12nm_mean

array([0.24162306, 0.24855835, 0.15340939, 0.17919348, 0.17721573])

In [43]:
fc_12nm_std = np.array([np.std(fcnt_rg1_2nm['Negative']), np.std(fcnt_rg1_2nm['Positive'])
                       ,np.std(fcnt_rg1_2nm['Polar']),np.std(fcnt_rg1_2nm['Hydrophobic'])
                       , np.std(fcnt_rg1_2nm['Aromatic'])])
fc_12nm_std

array([0.03083888, 0.05550695, 0.04379065, 0.05910623, 0.10279926])

In [120]:
x_pos = np.arange(5)
aa_types = ["Negative", "Positive", "Polar", "Hydrophobic", "Aromatic"]
fig = plt.figure(figsize=(7,7))
fig.canvas.layout.width = '500px'
fig.canvas.layout.height = '400px'
plt.bar(x_pos, fc_12nm_mean, yerr=fc_12nm_std, ecolor='black',capsize=5, color='c')
plt.title(r'Fractional Contacts 1.2 nm Rg restrained', fontsize=15)
plt.xticks(x_pos, labels=aa_types, fontsize=12)
plt.ylabel(r'Fractional Contacts', fontsize=15)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Fractional Contacts')

### Total fraction of contacts: averages and std dev calc from 5 20ns blocks

In [45]:
# Average of total fraction of contacts
np.mean(fcnt_rg1_2nm['total_frac'])

0.09897330595482545

In [46]:
# Std Deviation of total fraction of contacts
np.std(fcnt_rg1_2nm['total_frac'])

0.026722461576997848

### Avg no. PLGA residues per BSA AA residue group 

In [ ]:
prgrp_1_2nm

In [ ]:
mean_12nm = np.zeros(shape=5)
std_12nm = np.zeros(shape=5)
count = 0
for key, value in prgrp_1_2nm.items():
    mpl_12nm = []
    var_12nm = []
    for i in prgrp_1_2nm[str(key)].flat:
        mpl_12nm.append(i[0])
        var_12nm.append((i[1])**2)
    
    # calc frac cont averages
    mean_12nm[count] = np.mean(mpl_12nm)
    
    # calc frac cont std dev: https://stats.stackexchange.com/questions/25848/how-to-sum-a-standard-deviation 
    std_12nm[count] = np.std(mpl_12nm)
    # std_12nm[count] = np.sqrt(np.sum(var_12nm)/5)
    
    count += 1


In [ ]:
mean_12nm

In [ ]:
std_12nm 

In [ ]:
x_pos = np.arange(5)
aa_types = ["Negative", "Positive", "Polar", "Hydrophobic", "Aromatic"]
plt.figure(figsize=(7,7))
plt.bar(x_pos, mean_12nm, yerr=std_12nm, ecolor='black',capsize=5)
plt.title(r'No. of PLGA residues 1.2 nm Rg restrained', fontsize=15)
plt.xticks(x_pos, labels=aa_types, fontsize=12)
plt.ylabel(r'No. of PLGA residues', fontsize=15)

### Protein/polymer contact map movie

In [ ]:
fig = plt.figure(figsize=(10,10))

# Set the axis and the plot titles pp

plt.title("BSA/PLGA contact map 1.2 nm res", fontsize=22, loc='left')
plt.xlabel("PLGA Residue No.", fontsize=22)
plt.ylabel("BSA Residue No.", fontsize=20)

 # Set the axis range 
plt.ylim(583, 0)
plt.xlim(0, 60)

# Plot bands for each chain 
BANDS = (
    (0, 20, "purple", "B"),
    (20, 40, "blue", "C"),
    (40, 60, "green", "D"),
)
    
text_y = 0.98 # Close to the top
for start, stop, color, band in BANDS:
    plt.axvspan(start, stop,color=color, alpha=0.15)
    text_x = middle_of_band(start,stop)
    plt.text(
        text_x,
        text_y,
        "PLGA chain " + band,
        color=color,
        fontsize=18,
        transform=fig.gca().transAxes,
        horizontalalignment='center',
        verticalalignment='center',
        style='italic',
    )
    
plt.text(0.93, 1, "Time [ns]:", fontsize=20, transform=fig.gca().transAxes, horizontalalignment='right', verticalalignment='bottom')

# Set tick label size
fig.gca().tick_params(axis='both', which='major', labelsize=20)

ims = []
for i in range(10000):
    data = trj_ppmap_12nm[i]
    im = plt.imshow(data, aspect='auto', cmap='Greys')
    t_sim = plt.text(1, 1, str(i/100), fontsize=20, transform=fig.gca().transAxes, horizontalalignment='right', verticalalignment='bottom')
    ims.append([im, t_sim])
    
ani = animation.ArtistAnimation(fig, ims, blit=True, repeat=False)
ani.save('1.2nm_res.mp4',writer='ffmpeg', fps=50, bitrate=100000)
#plt.tight_layout()
#plt.show()

In [ ]:
def com_plga_oligs(g2_atoms, n_mon):
    hj_1 = []
    hj_2 = []
    hj_3 = []
    for i in range(n_mon):
        if i == 0:
            absdif = np.abs(np.diff(np.equal(list(g2_atoms.resids),i+1).view(np.int8)))
            rep_pga = np.concatenate(([0], np.where(absdif == 1)[0]))
            rep_pga += 1
            #print(rep_pga)
            rpn20 = rep_pga.reshape(3,2)
            hj_1.append(g2_atoms[rpn20[0][0]:rpn20[0][1]].center_of_mass())
            hj_2.append(g2_atoms[rpn20[1][0]:rpn20[1][1]].center_of_mass())
            hj_3.append(g2_atoms[rpn20[2][0]:rpn20[2][1]].center_of_mass())
        elif i != 0 and i != 19:
            absdif = np.abs(np.diff(np.equal(list(g2_atoms.resids),i+1).view(np.int8)))
            rep_pga = np.where(absdif == 1)[0]+1
            #print(rep_pga)
            rpn20 = rep_pga.reshape(3,2)
            hj_1.append(g2_atoms[rpn20[0][0]:rpn20[0][1]].center_of_mass())
            hj_2.append(g2_atoms[rpn20[1][0]:rpn20[1][1]].center_of_mass())
            hj_3.append(g2_atoms[rpn20[2][0]:rpn20[2][1]].center_of_mass())
        elif i == 19: 
            absdif = np.abs(np.diff(np.equal(list(g2_atoms.resids),i+1).view(np.int8)))
            rep_pga = np.where(absdif == 1)[0]+1
            rep_pga = np.concatenate((rep_pga,[len(g2_atoms)]))
            #print(rep_pga)
            rpn20 = rep_pga.reshape(3,2)
            hj_1.append(g2_atoms[rpn20[0][0]:rpn20[0][1]].center_of_mass())
            hj_2.append(g2_atoms[rpn20[1][0]:rpn20[1][1]].center_of_mass())
            hj_3.append(g2_atoms[rpn20[2][0]:rpn20[2][1]].center_of_mass())
    oligs_cb = np.concatenate((np.array(hj_1), np.array(hj_2), np.array(hj_3)))
    
    return oligs_cb

In [ ]:
cm_12nmoligs = com_plga_oligs(all_pn20, 20)
#cm_12nmoligs

In [ ]:
len(cm_12nmoligs)

# 1.5 nm PLGA restrained Rg 100 ns trajectory

Load the rg = 1.5 nm (3 PLGA N = 20 oligomer/BSA system) 

In [47]:
# Set up the MD Simulation
u15nm_pn20 = mda.Universe("../1.5nm_bsa_prod/k4000_1.5nm/res_1.5nmconf.pdb"
                      , "../1.5nm_bsa_prod/k4000_1.5nm/1_5nmppnopbc.xtc")

In [48]:
u15nm_pn20

<Universe with 10148 atoms>

In [49]:
pn20_len15 = len(u15nm_pn20.trajectory)
pn20_len15

10001

In [50]:
#Select all the PLGA residues, heavy atoms only 
all15_pn20 = u15nm_pn20.select_atoms("resname sPLG PLG tPLG and not type H")
all15_pn20

<AtomGroup with 543 atoms>

In [51]:
# Select BSA residues, heavy atoms only 
prot_15nm = u15nm_pn20.select_atoms("protein and not type H")
prot_15nm

<AtomGroup with 4653 atoms>

### Contact Analysis

In [52]:
#dmax = 4.0, protein group(4653 atoms), plga atom group (543 atoms), took 381.6 s (6 min 36s on 4 cores)
start = 0
end = pn20_len15 - 1
s_time = timeit.default_timer()
h2di_15nm = aa_frmcount(prot_15nm, all15_pn20, dmax, u15nm_pn20, start, end)
timeit.default_timer() - s_time
h2di_15nm

{'<Residue ASP, 1>': [82.0, 0.0082],
 '<Residue THR, 2>': [557.0, 0.0557],
 '<Residue HIS, 3>': [517.0, 0.0517],
 '<Residue LYS, 4>': [650.0, 0.065],
 '<Residue GLU, 6>': [38.0, 0.0038],
 '<Residue ARG, 10>': [33.0, 0.0033],
 '<Residue PHE, 11>': [1.0, 0.0001],
 '<Residue LYS, 12>': [26.0, 0.0026],
 '<Residue GLU, 16>': [1.0, 0.0001],
 '<Residue ASP, 37>': [1.0, 0.0001],
 '<Residue VAL, 40>': [1.0, 0.0001],
 '<Residue LYS, 41>': [10.0, 0.001],
 '<Residue ASN, 44>': [8.0, 0.0008],
 '<Residue GLU, 45>': [93.0, 0.0093],
 '<Residue GLU, 48>': [2.0, 0.0002],
 '<Residue PHE, 49>': [101.0, 0.0101],
 '<Residue LYS, 51>': [3.0, 0.0003],
 '<Residue ALA, 55>': [4.0, 0.0004],
 '<Residue ASP, 56>': [15.0, 0.0015],
 '<Residue GLU, 57>': [410.0, 0.041],
 '<Residue SER, 58>': [405.0, 0.0405],
 '<Residue HIS, 59>': [200.0, 0.02],
 '<Residue ALA, 60>': [1052.0, 0.1052],
 '<Residue GLY, 61>': [878.0, 0.0878],
 '<Residue GLU, 63>': [896.0, 0.0896],
 '<Residue LYS, 64>': [2046.0, 0.2046],
 '<Residue SER, 6

In [53]:
len(h2di_15nm.keys())

183

In [54]:
pr_res15nm = list(prot_15nm.residues)
ss_res15nm = [str(row) for row in pr_res15nm]
rkg_15nm = {key:h2di_15nm[key][0] for key, value in h2di_15nm.items()}
plg_1_5nmaa = pd.DataFrame(data=ss_res15nm, columns=["BSA_des_res"])
plg_1_5nmaa['mda_plga_frm_1.5nm'] = plg_1_5nmaa['BSA_des_res'].map(rkg_15nm)
plg_1_5nmaa['BSA_des_res'] = red_bsa
plg_1_5nmaa['mda_plga_frm_1.5nm'] = plg_1_5nmaa['mda_plga_frm_1.5nm'].replace('nan', np.nan).fillna(0)
plg_1_5nmaa.head()

,BSA_des_res,mda_plga_frm_1.5nm
0,ASP 1,82.0
1,THR 2,557.0
2,HIS 3,517.0
3,LYS 4,650.0
4,SER 5,0.0


In [56]:
# Extract mean occupancy values
pr_res_15nm = list(prot_15nm.residues)
ss_res_15nm = [str(row) for row in pr_res_15nm]
rkg_15nm = {key:h2di_15nm[key][1] for key, value in h2di_15nm.items()}
plg_1_2nm_occ['mda_occ_1.5nm'] = plg_1_2nm_occ['BSA_des_res'].map(rkg_15nm)
plg_1_2nm_occ['mda_occ_1.5nm'] = plg_1_2nm_occ['mda_occ_1.5nm'].replace('nan', np.nan).fillna(0)
plg_1_2nm_occ['mda_occ_1.5nm'] = plg_1_2nm_occ['mda_occ_1.5nm'].round(2)
plg_1_2nm_occ

,BSA_des_res,mda_occ_1.2nm,mda_occ_1.5nm
0,"<Residue ASP, 1>",0.00,0.01
1,"<Residue THR, 2>",0.00,0.06
2,"<Residue HIS, 3>",0.00,0.05
3,"<Residue LYS, 4>",0.00,0.06
4,"<Residue SER, 5>",0.00,0.00
...,...,...,...
578,"<Residue GLN, 579>",0.00,0.04
579,"<Residue THR, 580>",0.00,0.28
580,"<Residue ALA, 581>",0.01,0.31
581,"<Residue LEU, 582>",0.00,0.10


In [57]:
bsa_r = np.array(list(prot_15nm.resids)) # shape is 4652
m_occ_15nm = np.array(list(plg_1_2nm_occ['mda_occ_1.5nm'])) # shape is 583
m_occ_15New = np.zeros(shape=(4653))
at_ind = np.where(bsa_r[:-1] != bsa_r[1:])[0]
at_in_nw = np.sort(np.append([0,4653],at_ind))
nw_v = 0
for i in range(583):
    b = at_in_nw[i+1] +1
    m_occ_15New[nw_v:b] = m_occ_15nm[i]
    nw_v = at_in_nw[i+1] + 1 

In [58]:
m_occ_15New[0:33]

array([0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.06, 0.06, 0.06,
       0.06, 0.06, 0.06, 0.06, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05,
       0.05, 0.05, 0.05, 0.06, 0.06, 0.06, 0.06, 0.06, 0.06, 0.06, 0.06])

In [ ]:
prot_15nm.occupancies[3089:3099]

In [ ]:
list(prot_15nm.atoms[3089:3099])

In [ ]:
#np.nonzero(m_occ)

### Visualize Occupanct on protein 

In [ ]:
prot_15nm.occupancies = m_occ_15New
prot_15nm.occupancies[0:33]

In [ ]:
with mda.Writer("prot_15nmresNEEW.pdb") as pdb:
    pdb.write(prot_15nm)

### Residue Importance: 1.5 nm restrained 

In [ ]:
# Need to fix function, the residue number are not counting the other 2 PLGA oligomers cuz of same resid number
trj_ppmap_15nm = prot_poly_cntmovie(prot_15nm, all15_pn20, dmax, u15nm_pn20, 0, 10000)
#trj_ppmap_12nm_chC = prot_poly_cntmovie(prot, all_pn20_C, dmax, u_pn20, 0, 10000)

In [59]:
trj_ppmap_15nm = np.load("1.5nm_res.npy", allow_pickle=True)

In [ ]:
np.save('1.5nm_res.npy', trj_ppmap_15nm)    # .npy extension is added if not given

In [60]:
trj_ppmap_15nm[0].shape

(583, 60)

In [61]:
np.sum(trj_ppmap_15nm[1000][0])

0.0

In [62]:
kj = np.zeros(shape=(10000, 583))
kj[:,582].shape

(10000,)

In [63]:
pp_15nm_ct = np.zeros(shape=(10000, 583))
for i in range(10000):
    for j in range(583):
        pp_15nm_ct[i][j] = np.sum(trj_ppmap_15nm[i][j])

In [64]:
pp_15nmtot = np.zeros(shape=(583))
for i in range(583):
    pp_15nmtot[i] = np.sum(pp_15nm_ct[:,i])
#pp_12nmtot

In [65]:
np.nonzero(pp_15nmtot)

(array([  0,   1,   2,   3,   5,   9,  10,  11,  39,  40,  44,  48,  50,
         54,  55,  56,  57,  58,  59,  60,  62,  63,  64,  67,  68,  70,
         71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  87,  88,
         90,  91,  92,  93,  94,  95,  96,  97, 112, 113, 114, 115, 116,
        117, 118, 119, 120, 126, 127, 128, 131, 143, 164, 165, 166, 168,
        169, 170, 171, 172, 173, 174, 175, 177, 178, 179, 181, 182, 184,
        185, 186, 188, 189, 206, 234, 238, 241, 242, 250, 254, 257, 272,
        274, 297, 298, 299, 376, 380, 384, 387, 388, 390, 391, 392, 393,
        394, 395, 396, 397, 398, 399, 400, 402, 403, 404, 408, 423, 426,
        427, 430, 432, 433, 434, 436, 437, 438, 439, 440, 441, 442, 443,
        450, 508, 509, 510, 512, 513, 514, 515, 516, 517, 518, 519, 520,
        521, 536, 538, 539, 540, 541, 543, 544, 545, 546, 547, 548, 550,
        551, 554, 555, 557, 558, 559, 560, 561, 562, 563, 569, 572, 573,
        575, 576, 577, 578, 579, 580, 581, 582]),)

In [119]:
y_pos = np.arange(583) + 1
wid = np.zeros(shape=583)
wid += 1.5
#wid
fig = plt.figure(figsize=(12,12))
fig.canvas.layout.width = '800px'
fig.canvas.layout.height = '700px'
plt.bar(y_pos+0.25, pp_15nmtot, align='center',width=wid, color='#562A8B', alpha=0.3, label='1.5 nm PLGA')
plt.bar(y_pos, pp_12nmtot, align='center',width=wid, alpha=0.5, color='#1D77CF',label='1.2 nm PLGA')
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.xlim([0,600])
plt.ylim([0,50000])
plt.legend(fontsize=14)
plt.ylabel(r'Total No. of PLGA contacts', fontsize=15)
plt.xlabel(r'BSA Residue ID', fontsize=15)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'BSA Residue ID')

### Total number of residues that are within 4 angstroms of a PLGA oligomer residue within a 100 ns trajectory block

In [67]:
apl_15nm = []

# Some residues don't have any contact with the 3 N = 20 PLGA oligomers within 100 ns,
# Put residues that do have contact with BSA in a separate list
for index, r_pl in plg_1_5nmaa.iterrows():
    if r_pl['mda_plga_frm_1.5nm'] != 0:
        apl_15nm.append(r_pl['BSA_des_res'])
        
# This chunk of code gets an AA count from the above list, in order 
# to get a total number of residues that contact BSA
cpl_15nm = []

for index, r_a in aa_count.iterrows():
    count = 0
    for i in range(len(apl_15nm)):
        if r_a['Amino_acids'] in apl_15nm[i]:
            count += 1
    cpl_15nm.append(count)      
        
aa_count['plga_1.5nm_100ns'] = cpl_15nm
#aa_count.drop('No_of_surf_res (VMD)', axis=1, inplace=True)
aa_count

,Amino_acids,No_of_surf_res (MDAnalysis),No_of_surf_res (VMD),plga_1.2nm_100ns,plga_1.5nm_100ns
0,LYS,59,59,18,29
1,ARG,23,23,5,10
2,HIS,16,16,2,3
3,ASP,40,40,12,18
4,GLU,59,59,16,30
5,SER,23,23,3,6
6,THR,29,29,9,14
7,CYS,24,24,7,8
8,TYR,19,19,2,2
9,ASN,14,14,4,6


In [68]:
# This gives the total number of residues that are within 4 angstroms of a PLGA oligomer residue
# within a 100 ns trajectory block
aa_count['plga_1.5nm_100ns'].sum()

183

In [69]:
# This gives the total number of residues that are within 4 angstroms of a water molecule
# within a 1 ns trajectory block
aa_count['No_of_surf_res (MDAnalysis)'].sum()

487

In [70]:
# This gives the total fraction of contacts within the 1.2 nm Rg 100 ns trajectory
aa_count['plga_1.5nm_100ns'].sum()/aa_count['No_of_surf_res (MDAnalysis)'].sum()

0.3757700205338809

In [71]:
# Mean occupancy and std deviation 
ll_mo15 = [value[1] for key, value in h2di_15nm.items()]
print("Mean Occpancy (1.5 nm Rg): "+str(np.mean(ll_mo15)), "Occ. std. dev.: "+str(np.std(ll_mo15)))

Mean Occpancy (1.5 nm Rg): 0.1732382513661202 Occ. std. dev.: 0.25491500578855153


In [72]:
cd_15nm = frac_cont(h2di_15nm)
cd_15nm

{'Negative': [77109.0, 48, 293978.0625, 0.19755475499091144],
 'Positive': [63557.0, 42, 276926.9285714286, 0.18609630616336958],
 'Polar': [87653.0, 45, 356455.5333333333, 0.23953993353780037],
 'Hydrophobic': [88707.0, 48, 338195.4375, 0.22726905615399992],
 'Aromatic': [9728.0, 8, 222528.0, 0.14953994915391872]}

### Calc. fractional contacts for each AA group type 

In [73]:
fcnt_rg1_5nm, prgrp_1_5nm, aa_matx_15nm = bavg_frac_cnt(5, prot_15nm, all15_pn20, dmax,
                                                        u15nm_pn20, no_surf, 0, 10000)

0
2000
2000
4000
4000
6000
6000
8000
8000
10000


In [74]:
fcnt_rg1_5nm

{'Negative': array([0.21702984, 0.21582487, 0.21504858, 0.19375128, 0.20812885]),
 'Positive': array([0.18445364, 0.22567269, 0.17180313, 0.18314279, 0.19330487]),
 'Polar': array([0.35721667, 0.19655431, 0.24416846, 0.22038798, 0.22895071]),
 'Hydrophobic': array([0.17792102, 0.22046298, 0.22752588, 0.19358892, 0.21477886]),
 'Aromatic': array([0.06337883, 0.14148514, 0.14145395, 0.20912903, 0.15483672]),
 'total_frac': array([0.16632444, 0.22176591, 0.22792608, 0.20533881, 0.22176591])}

In [75]:
fc_15nm_mean = np.array([np.mean(fcnt_rg1_5nm['Negative']), np.mean(fcnt_rg1_5nm['Positive'])
                        ,np.mean(fcnt_rg1_5nm['Polar']),np.mean(fcnt_rg1_5nm['Hydrophobic'])
                        , np.mean(fcnt_rg1_5nm['Aromatic'])])
fc_15nm_mean

array([0.20995669, 0.19167542, 0.24945563, 0.20685553, 0.14205673])

In [76]:
fc_15nm_std = np.array([np.std(fcnt_rg1_5nm['Negative']), np.std(fcnt_rg1_5nm['Positive'])
                       ,np.std(fcnt_rg1_5nm['Polar']),np.std(fcnt_rg1_5nm['Hydrophobic'])
                       , np.std(fcnt_rg1_5nm['Aromatic'])])
fc_15nm_std

array([0.00867581, 0.01832123, 0.05604342, 0.01837615, 0.04658954])

In [118]:
x_pos = np.arange(5)
width = 0.35
aa_types = ["Negative", "Positive", "Polar", "Hydrophobic", "Aromatic"]
plt.figure(figsize=(7,7))
plt.bar(x_pos, fc_12nm_mean, width, yerr=fc_12nm_std, ecolor='black',capsize=5, color='royalblue')
plt.bar(x_pos+width, fc_15nm_mean, width, yerr=fc_15nm_std, ecolor='black',capsize=5, color='c')
plt.title(r'Fractional Contacts Rg restrained', fontsize=15)
plt.xticks(x_pos+width/2, labels=aa_types, fontsize=12)
plt.legend(['Rg = 1.2 nm', 'Rg = 1.5 nm'], frameon=False)
plt.ylabel(r'Fractional Contacts', fontsize=15)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Fractional Contacts')

### Total fraction of contacts: averages and std dev calc from 5 20 ns blocks

In [78]:
np.mean(fcnt_rg1_5nm['total_frac'])

0.2086242299794661

In [79]:
np.std(fcnt_rg1_5nm['total_frac'])

0.022441192759448465

In [ ]:
prgrp_1_5nm

In [ ]:
# matrix containing the avg number of PLGA residues for each block for each amino acid 
np.where(aa_matx_15nm[0, 0] != 0)

### Avg no. PLGA residues per BSA AA residue group 

In [ ]:
prgrp_1_5nm

In [ ]:
mean_15nm = np.zeros(shape=5)
std_15nm = np.zeros(shape=5)
count = 0
for key, value in prgrp_1_5nm.items():
    mpl_15nm = []
    var_15nm = []
    for i in prgrp_1_5nm[str(key)].flat:
        mpl_15nm.append(i[0])
        var_15nm.append((i[1])**2)
    
    # calc frac cont averages
    mean_15nm[count] = np.mean(mpl_15nm)
    
    # calc frac cont std dev: https://stats.stackexchange.com/questions/25848/how-to-sum-a-standard-deviation 
    std_15nm[count] = np.std(mpl_15nm)
    #std_15nm[count] = np.sqrt(np.sum(var_15nm)/5)
    
    count += 1


In [ ]:
mean_15nm

In [ ]:
std_15nm

In [ ]:
x_pos = np.arange(5)
width = 0.35
aa_types = ["Negative", "Positive", "Polar", "Hydrophobic", "Aromatic"]
plt.figure(figsize=(7,7))
plt.bar(x_pos, mean_12nm, width, yerr=std_12nm, ecolor='black',capsize=5, color='royalblue')
plt.bar(x_pos+width, mean_15nm, width, yerr=std_15nm, ecolor='black',capsize=5, color='c')
plt.title(r'No. of PLGA residues Rg restrained', fontsize=15)
plt.xticks(x_pos+width/2, labels=aa_types, fontsize=12)
plt.legend(['Rg = 1.2 nm', 'Rg = 1.5 nm'], frameon=False)
plt.ylabel(r'No. of PLGA residues', fontsize=15)

### Protein/polymer contact map movie

In [ ]:
fig = plt.figure(figsize=(10,10))

# Set the axis and the plot titles pp

plt.title("BSA/PLGA contact map 1.5 nm res.", fontsize=22, loc='left')
plt.xlabel("PLGA Residue No.", fontsize=22)
plt.ylabel("BSA Residue No.", fontsize=20)

 # Set the axis range 
plt.ylim(583, 0)
plt.xlim(0, 60)

# Plot bands for each chain 
BANDS = (
    (0, 20, "purple", "B"),
    (20, 40, "blue", "C"),
    (40, 60, "green", "D"),
)
    
text_y = 0.98 # Close to the top
for start, stop, color, band in BANDS:
    plt.axvspan(start, stop,color=color, alpha=0.15)
    text_x = middle_of_band(start,stop)
    plt.text(
        text_x,
        text_y,
        "PLGA chain " + band,
        color=color,
        fontsize=18,
        transform=fig.gca().transAxes,
        horizontalalignment='center',
        verticalalignment='center',
        style='italic',
    )
    
plt.text(0.94, 1, "Time [ns]:", fontsize=20, transform=fig.gca().transAxes, horizontalalignment='right', verticalalignment='bottom')

# Set tick label size
fig.gca().tick_params(axis='both', which='major', labelsize=20)

ims = []
for i in range(10000):
    data = trj_ppmap_15nm[i]
    im = plt.imshow(data, aspect='auto', cmap='Greys')
    t_sim = plt.text(1.03, 1, str(i/100), fontsize=20, transform=fig.gca().transAxes, horizontalalignment='right', verticalalignment='bottom')
    ims.append([im, t_sim])
    
ani = animation.ArtistAnimation(fig, ims, blit=True, repeat=False)
ani.save('1.5nm_res.mp4', writer='ffmpeg', fps=50, bitrate=100000)
#plt.tight_layout()
#plt.show()

# 2 nm restrained Rg PLGA 100 ns trajectory

Load the rg = 1.5 nm (3 PLGA N = 20 oligomer/BSA system) 

### Contact Analyis

In [80]:
# Set up the MD Simulation
u2nm_pn20 = mda.Universe("../2nm_bsa_prod/k4000_2nm/new_conf2nm.pdb"
                      , "../2nm_bsa_prod/k4000_2nm/nopbc_ppsys.xtc")

In [81]:
u2nm_pn20

<Universe with 10148 atoms>

In [82]:
pn20_len2nm = len(u2nm_pn20.trajectory)
pn20_len2nm

10001

In [83]:
#Select all the PLGA residues, heavy atoms only 
all2nm_pn20 = u2nm_pn20.select_atoms("resname sPLG PLG tPLG and not type H")
all2nm_pn20

<AtomGroup with 543 atoms>

In [84]:
# Select BSA residues, heavy atoms only 
prot_2nm = u2nm_pn20.select_atoms("protein and not type H")
prot_2nm

<AtomGroup with 4653 atoms>

In [85]:
#dmax = 4.0, protein group(4653 atoms), plga atom group (543 atoms), took 381.6 s (6 min 36s on 4 cores)
start = 0
end = pn20_len2nm - 1
s_time = timeit.default_timer()
h2di_2nm = aa_frmcount(prot_2nm, all2nm_pn20, dmax, u2nm_pn20, start, end)
timeit.default_timer() - s_time
h2di_2nm

{'<Residue ASP, 1>': [55.0, 0.0055],
 '<Residue THR, 2>': [105.0, 0.0105],
 '<Residue HIS, 3>': [180.0, 0.018],
 '<Residue LYS, 4>': [24.0, 0.0024],
 '<Residue HIS, 9>': [58.0, 0.0058],
 '<Residue PHE, 11>': [862.0, 0.0862],
 '<Residue LYS, 12>': [1051.0, 0.1051],
 '<Residue ASP, 13>': [364.0, 0.0364],
 '<Residue LEU, 14>': [138.0, 0.0138],
 '<Residue GLY, 15>': [326.0, 0.0326],
 '<Residue GLU, 16>': [1092.0, 0.1092],
 '<Residue GLU, 17>': [421.0, 0.0421],
 '<Residue HIS, 18>': [12.0, 0.0012],
 '<Residue LYS, 20>': [443.0, 0.0443],
 '<Residue PRO, 35>': [2.0, 0.0002],
 '<Residue PHE, 36>': [3.0, 0.0003],
 '<Residue ASP, 37>': [1360.0, 0.136],
 '<Residue GLU, 38>': [572.0, 0.0572],
 '<Residue VAL, 40>': [1464.0, 0.1464],
 '<Residue LYS, 41>': [1661.0, 0.1661],
 '<Residue ASN, 44>': [1523.0, 0.1523],
 '<Residue GLU, 45>': [247.0, 0.0247],
 '<Residue THR, 47>': [1.0, 0.0001],
 '<Residue GLU, 48>': [3062.0, 0.3062],
 '<Residue PHE, 49>': [12.0, 0.0012],
 '<Residue LYS, 51>': [4150.0, 0.415

In [86]:
len(h2di_2nm.keys())

192

In [87]:
pr_res2nm = list(prot_2nm.residues)
ss_res2nm = [str(row) for row in pr_res2nm]
rkg_2nm = {key:h2di_2nm[key][0] for key, value in h2di_2nm.items()}
plg_2nmaa = pd.DataFrame(data=ss_res2nm, columns=["BSA_des_res"])
plg_2nmaa['mda_plga_frm_2nm'] = plg_2nmaa['BSA_des_res'].map(rkg_2nm)
plg_2nmaa['BSA_des_res'] = red_bsa
plg_2nmaa['mda_plga_frm_2nm'] = plg_2nmaa['mda_plga_frm_2nm'].replace('nan', np.nan).fillna(0)
plg_2nmaa.head()

,BSA_des_res,mda_plga_frm_2nm
0,ASP 1,55.0
1,THR 2,105.0
2,HIS 3,180.0
3,LYS 4,24.0
4,SER 5,0.0


In [122]:
# Extract mean occupancy values
pr_res_2ur = list(prot_2nm.residues)
ss_res_2ur = [str(row) for row in pr_res_2ur]
rkg_2ur = {key:h2di_2nm[key][1] for key, value in h2di_2nm.items()}
plg_1_2nm_occ['mda_occ_2nm'] = plg_1_2nm_occ['BSA_des_res'].map(rkg_2ur)
plg_1_2nm_occ['mda_occ_2nm'] = plg_1_2nm_occ['mda_occ_2nm'].replace('nan', np.nan).fillna(0)
plg_1_2nm_occ['mda_occ_2nm'] = plg_1_2nm_occ['mda_occ_2nm'].round(2)
plg_1_2nm_occ

,BSA_des_res,mda_occ_1.2nm,mda_occ_1.5nm,mda_occ_2nm
0,"<Residue ASP, 1>",0.00,0.01,0.01
1,"<Residue THR, 2>",0.00,0.06,0.01
2,"<Residue HIS, 3>",0.00,0.05,0.02
3,"<Residue LYS, 4>",0.00,0.06,0.00
4,"<Residue SER, 5>",0.00,0.00,0.00
...,...,...,...,...
578,"<Residue GLN, 579>",0.00,0.04,0.58
579,"<Residue THR, 580>",0.00,0.28,0.38
580,"<Residue ALA, 581>",0.01,0.31,0.01
581,"<Residue LEU, 582>",0.00,0.10,0.03


In [123]:
bsa_r = np.array(list(prot.resids)) # shape is 4652
m_occ_2nm = np.array(list(plg_1_2nm_occ['mda_occ_2nm'])) # shape is 583
m_occ_2New = np.zeros(shape=(4653))
at_ind = np.where(bsa_r[:-1] != bsa_r[1:])[0]
at_in_nw = np.sort(np.append([0,4653],at_ind))
nw_v = 0
for i in range(583):
    b = at_in_nw[i+1] +1
    m_occ_2New[nw_v:b] = m_occ_2nm[i]
    nw_v = at_in_nw[i+1] + 1 

In [124]:
np.nonzero(m_occ_2New)

(array([   0,    1,    2, ..., 4650, 4651, 4652]),)

### Visualize Occupanct on protein 

In [125]:
prot_2nm.occupancies = m_occ_2New
prot_2nm.occupancies

array([0.01, 0.01, 0.01, ..., 0.23, 0.23, 0.23])

In [126]:
with mda.Writer("prot_2nmres.pdb") as pdb:
    pdb.write(prot_2nm)

### Residue Importance: 2 nm restrained 

In [ ]:
# Need to fix function, the residue number are not counting the other 2 PLGA oligomers cuz of same resid number
trj_ppmap_2nm = prot_poly_cntmovie(prot_2nm, all2nm_pn20, dmax, u2nm_pn20, 0, 10000)
#trj_ppmap_12nm_chC = prot_poly_cntmovie(prot, all_pn20_C, dmax, u_pn20, 0, 10000)

In [ ]:
np.save('2nm_res.npy', trj_ppmap_2nm)    # .npy extension is added if not given

In [92]:
trj_ppmap_2nm = np.load('2nm_res.npy', allow_pickle=True)
trj_ppmap_2nm

array([array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]),
       array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]),
       array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]),
       ...,
       array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0.,

In [93]:
trj_ppmap_2nm[0].shape

(583, 60)

In [94]:
np.sum(trj_ppmap_2nm[1000][0])

0.0

In [95]:
kj = np.zeros(shape=(10000, 583))
kj[:,582].shape

(10000,)

In [96]:
pp_2nm_ct = np.zeros(shape=(10000, 583))
for i in range(10000):
    for j in range(583):
        pp_2nm_ct[i][j] = np.sum(trj_ppmap_2nm[i][j])

In [97]:
pp_2nmtot = np.zeros(shape=(583))
for i in range(583):
    pp_2nmtot[i] = np.sum(pp_2nm_ct[:,i])
#pp_12nmtot

In [98]:
np.nonzero(pp_2nmtot)

(array([  0,   1,   2,   3,   8,  10,  11,  12,  13,  14,  15,  16,  17,
         19,  34,  35,  36,  37,  39,  40,  43,  44,  46,  47,  48,  50,
         51,  52,  53,  54,  55,  56,  57,  58,  59,  75,  76,  77,  78,
         85,  88, 117, 119, 120, 122, 123, 125, 126, 127, 128, 129, 130,
        131, 157, 158, 161, 163, 164, 165, 166, 167, 168, 169, 170, 171,
        172, 174, 175, 178, 179, 223, 224, 225, 253, 260, 263, 264, 265,
        266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278,
        279, 280, 281, 282, 283, 284, 287, 290, 291, 292, 293, 294, 295,
        296, 297, 298, 299, 300, 332, 334, 335, 336, 337, 338, 339, 350,
        357, 372, 373, 374, 375, 376, 377, 378, 379, 380, 381, 383, 384,
        386, 387, 388, 390, 391, 393, 394, 395, 396, 397, 412, 415, 432,
        436, 437, 438, 439, 440, 441, 442, 443, 444, 445, 469, 481, 484,
        485, 487, 488, 489, 490, 492, 493, 494, 495, 496, 497, 498, 499,
        500, 501, 502, 503, 533, 534, 535, 536, 541

In [138]:
a = np.sum(pp_2nmtot)

In [139]:
b = np.sum(pp_15nmtot)

In [140]:
c = np.sum(pp_12nmtot)

In [142]:
plt.close('all')

In [147]:
y_pos = np.arange(583) + 1
wid = np.zeros(shape=583)
wid += 1.5
#wid
fig = plt.figure(figsize=(12,12))
fig.canvas.layout.width = '800px'
fig.canvas.layout.height = '700px'
plt.bar(y_pos, pp_12nmtot/c, align='center',width=wid, alpha=0.5, color='#1D77CF',label='1.2 nm PLGA')
plt.bar(y_pos+0.25, pp_15nmtot/b, align='center',width=wid, color='#562A8B', alpha=0.3, label='1.5 nm PLGA')
plt.bar(y_pos+0.3, pp_2nmtot/a, align='center',width=wid, color='#4E4C4D', alpha=0.3, label='2 nm PLGA')
#plt.bar(y_pos+0.25, pp_15nmtot/b, align='center',width=wid, color='#562A8B', alpha=0.3, label='1.5 nm PLGA')
#plt.bar(y_pos, pp_12nmtot, align='center',width=wid, alpha=0.5, color='#1D77CF',label='1.2 nm PLGA')
plt.title("BSA in water with PLGA restrained, 100 ns", fontsize=18)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.xlim([0,600])
plt.ylim([0,0.2])
plt.legend(fontsize=14)
plt.ylabel(r'Normalized Total No. of PLGA contacts', fontsize=15)
plt.xlabel(r'BSA Residue ID', fontsize=15)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'BSA Residue ID')

### Total number of residues that are within 4 angstroms of a PLGA oligomer residue within a 100 ns trajectory block

In [100]:
apl_2nm = []

# Some residues don't have any contact with the 3 N = 20 PLGA oligomers within 100 ns,
# Put residues that do have contact with BSA in a separate list
for index, r_pl in plg_2nmaa.iterrows():
    if r_pl['mda_plga_frm_2nm'] != 0:
        apl_2nm.append(r_pl['BSA_des_res'])
        
# This chunk of code gets an AA count from the above list, in order 
# to get a total number of residues that contact BSA
cpl_2nm = []

for index, r_a in aa_count.iterrows():
    count = 0
    for i in range(len(apl_2nm)):
        if r_a['Amino_acids'] in apl_2nm[i]:
            count += 1
    cpl_2nm.append(count)      
        
aa_count['plga_2nm_100ns'] = cpl_2nm
#aa_count.drop('No_of_surf_res (VMD)', axis=1, inplace=True)
aa_count

,Amino_acids,No_of_surf_res (MDAnalysis),No_of_surf_res (VMD),plga_1.2nm_100ns,plga_1.5nm_100ns,plga_2nm_100ns
0,LYS,59,59,18,29,30
1,ARG,23,23,5,10,3
2,HIS,16,16,2,3,7
3,ASP,40,40,12,18,20
4,GLU,59,59,16,30,26
5,SER,23,23,3,6,7
6,THR,29,29,9,14,11
7,CYS,24,24,7,8,12
8,TYR,19,19,2,2,3
9,ASN,14,14,4,6,8


In [101]:
# This gives the total number of residues that are within 4 angstroms of a PLGA oligomer residue
# within a 100 ns trajectory block
aa_count['plga_2nm_100ns'].sum()

192

In [102]:
# This gives the total number of residues that are within 4 angstroms of a water molecule
# within a 1 ns trajectory block
aa_count['No_of_surf_res (MDAnalysis)'].sum()

487

In [103]:
# This gives the total fraction of contacts within the 1.2 nm Rg 100 ns trajectory
aa_count['plga_2nm_100ns'].sum()/aa_count['No_of_surf_res (MDAnalysis)'].sum()

0.3942505133470226

In [104]:
# Mean occupancy and std deviation 
ll_mo2nm = [value[1] for key, value in h2di_2nm.items()]
print("Mean Occpancy (2 nm Rg): "+str(np.mean(ll_mo2nm)), "Occ. std. dev.: "+str(np.std(ll_mo2nm)))

Mean Occpancy (2 nm Rg): 0.157603125 Occ. std. dev.: 0.21080215050350182


In [105]:
cd_2nm = frac_cont(h2di_2nm)
cd_2nm

{'Negative': [76035.0, 46, 317363.47826086957, 0.2125303739220412],
 'Positive': [76038.0, 40, 364982.39999999997, 0.24441957332974013],
 'Polar': [78944.0, 49, 309331.59183673473, 0.20715162071964027],
 'Hydrophobic': [71581.0, 57, 241114.94736842104, 0.16146864221182233],
 'Aromatic': [24419.0, 18, 260469.33333333334, 0.1744297898167561]}

### Calc. fractional contacts for each AA group type 

In [106]:
fcnt_rg2nm, prgrp_2nm, aa_matx_2nm = bavg_frac_cnt(5, prot_2nm, all2nm_pn20, dmax, u2nm_pn20, no_surf, 0, 10000)

0
2000
2000
4000
4000
6000
6000
8000
8000
10000


In [107]:
fcnt_rg2nm

{'Negative': array([0.16720553, 0.26918719, 0.16398971, 0.18059591, 0.19974406]),
 'Positive': array([0.26865875, 0.2000935 , 0.22166664, 0.21003575, 0.20974638]),
 'Polar': array([0.21738731, 0.21620981, 0.20630745, 0.22387009, 0.19534954]),
 'Hydrophobic': array([0.21733371, 0.1612945 , 0.21107994, 0.14886853, 0.16794913]),
 'Aromatic': array([0.12941469, 0.15321501, 0.19695626, 0.23662973, 0.22721088]),
 'total_frac': array([0.19507187, 0.2587269 , 0.22997947, 0.21560575, 0.23819302])}

In [108]:
fc_2nm_mean = np.array([np.mean(fcnt_rg2nm['Negative']), np.mean(fcnt_rg2nm['Positive'])
                        ,np.mean(fcnt_rg2nm['Polar']),np.mean(fcnt_rg2nm['Hydrophobic'])
                        , np.mean(fcnt_rg2nm['Aromatic'])])
fc_2nm_mean

array([0.19614448, 0.22204021, 0.21182484, 0.18130516, 0.18868531])

In [109]:
fc_2nm_std = np.array([np.std(fcnt_rg2nm['Negative']), np.std(fcnt_rg2nm['Positive'])
                       ,np.std(fcnt_rg2nm['Polar']),np.std(fcnt_rg2nm['Hydrophobic'])
                       , np.std(fcnt_rg2nm['Aromatic'])])
fc_2nm_std

array([0.0386299 , 0.02429128, 0.0099713 , 0.02762441, 0.04152696])

In [116]:
x_pos = np.arange(5)
width = 0.28
aa_types = ["Negative", "Positive", "Polar", "Hydrophobic", "Aromatic"]
plt.figure(figsize=(7,7))
plt.bar(x_pos, fc_12nm_mean, width, yerr=fc_12nm_std, ecolor='black',capsize=5, color='royalblue')
plt.bar(x_pos+width, fc_15nm_mean, width, yerr=fc_15nm_std, ecolor='black',capsize=5, color='c')
plt.bar(x_pos+(2*width), fc_2nm_mean, width, yerr=fc_2nm_std, ecolor='black',capsize=5, color='lightslategray')
plt.title(r'Fractional Contacts Rg restrained', fontsize=15)
plt.xticks(x_pos+width, labels=aa_types, fontsize=12)
plt.ylim(0,0.4)
plt.legend(['Rg = 1.2 nm', 'Rg = 1.5 nm', 'Rg = 2 nm'], frameon=False)
plt.ylabel(r'Fractional Contacts', fontsize=15)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Fractional Contacts')

### Total fraction of contacts: averages and std dev calc from 5 20 ns blocks

In [111]:
np.mean(fcnt_rg2nm['total_frac'])

0.22751540041067764

In [112]:
np.std(fcnt_rg2nm['total_frac'])

0.021394687930400173

In [113]:
prgrp_2nm

{'Negative': array([array([11.0775    ,  6.00378995]),
        array([19.3495    ,  5.01391561]),
        array([12.0635    ,  3.82053239]), array([17.3825   ,  5.1422946]),
        array([20.298     ,  4.90083626])], dtype=object),
 'Positive': array([array([18.093     ,  7.74431088]),
        array([20.6345    ,  4.94529167]),
        array([20.3625    ,  5.14772705]),
        array([19.0185    ,  4.37666057]),
        array([22.2625    ,  5.29118075])], dtype=object),
 'Polar': array([array([13.6235    ,  6.54505521]),
        array([22.808     ,  5.53869443]),
        array([21.084     ,  5.53876737]),
        array([18.1995    ,  5.02062743]),
        array([27.1795    ,  7.74217539])], dtype=object),
 'Hydrophobic': array([array([12.083     ,  6.26067976]),
        array([16.5775    ,  4.59923839]),
        array([19.1075    ,  6.33931729]),
        array([13.31      ,  4.01583117]),
        array([21.4955    ,  5.42217482])], dtype=object),
 'Aromatic': array([array([2.762     ,

In [115]:
# matrix containing the avg number of PLGA residues for each block for each amino acid 
np.where(aa_matx_2nm[0] != 0)

(array([  0,   1,   2,   8,  10,  11,  12,  13,  14,  15,  16,  17,  19,
         35,  36,  37,  39,  40,  43,  44,  47,  50,  51,  53,  54,  55,
         57,  58,  85,  88, 117, 119, 120, 122, 123, 125, 126, 127, 128,
        129, 130, 131, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172,
        174, 175, 179, 223, 253, 260, 263, 266, 267, 270, 271, 272, 273,
        274, 275, 277, 278, 279, 280, 281, 284, 292, 293, 294, 295, 296,
        298, 299, 337, 338, 390, 391, 393, 394, 395, 436, 437, 438, 439,
        440, 441, 442, 443]),)

### Avg no. PLGA residues per BSA AA residue group 

In [ ]:
prgrp_2nm

In [ ]:
mean_2nm = np.zeros(shape=5)
std_2nm = np.zeros(shape=5)
count = 0
for key, value in prgrp_2nm.items():
    mpl_2nm = []
    var_2nm = []
    for i in prgrp_2nm[str(key)].flat:
        mpl_2nm.append(i[0])
        var_2nm.append((i[1])**2)
    
    # calc frac cont averages
    mean_2nm[count] = np.mean(mpl_2nm)
    
    # calc frac cont std dev: https://stats.stackexchange.com/questions/25848/how-to-sum-a-standard-deviation 
    std_2nm[count] = np.std(mpl_2nm)
    #std_2nm[count] = np.sqrt(np.sum(var_2nm)/5)
    
    count += 1


In [ ]:
mean_2nm

In [ ]:
std_2nm

In [ ]:
mean_15nm

In [ ]:
std_15nm

In [ ]:
x_pos = np.arange(5)
width = 0.28
aa_types = ["Negative", "Positive", "Polar", "Hydrophobic", "Aromatic"]
plt.figure(figsize=(7,7))
plt.bar(x_pos, mean_12nm, width, yerr=std_12nm, ecolor='black',capsize=5, color='royalblue')
plt.bar(x_pos+width, mean_15nm, width, yerr=std_15nm, ecolor='black',capsize=5, color='c')
plt.bar(x_pos+(2*width), mean_2nm, width, yerr=std_2nm, ecolor='black',capsize=5, color='lightslategray')
plt.title(r'No. of PLGA residues Rg restrained', fontsize=15)
plt.xticks(x_pos+width, labels=aa_types, fontsize=12)
plt.legend(['Rg = 1.2 nm', 'Rg = 1.5 nm', 'Rg = 2 nm'], frameon=False)
plt.ylabel(r'No. of PLGA residues', fontsize=15)

### Protein/polymer contact map movie

In [ ]:
np.where(trj_load2nm_res[9920] == 0 )

In [ ]:
trj_load2nm_res[9920]

In [ ]:
trj_load2nm_res.shape

In [ ]:
fig = plt.figure(figsize=(10,10))

# Set the axis and the plot titles pp

plt.title("BSA/PLGA contact map 2 nm restrained", fontsize=22, loc='left')
plt.xlabel("PLGA Residue No.", fontsize=22)
plt.ylabel("BSA Residue No.", fontsize=20)

 # Set the axis range 
plt.ylim(583, 0)
plt.xlim(0, 60)

# Plot bands for each chain 
BANDS = (
    (0, 20, "purple", "B"),
    (20, 40, "blue", "C"),
    (40, 60, "green", "D"),
)
    
text_y = 0.98 # Close to the top
for start, stop, color, band in BANDS:
    plt.axvspan(start, stop,color=color, alpha=0.15)
    text_x = middle_of_band(start,stop)
    plt.text(
        text_x,
        text_y,
        "PLGA chain " + band,
        color=color,
        fontsize=18,
        transform=fig.gca().transAxes,
        horizontalalignment='center',
        verticalalignment='center',
        style='italic',
    )
    
plt.text(0.94, 1, "Time [ns]:", fontsize=20, transform=fig.gca().transAxes, horizontalalignment='right', verticalalignment='bottom')

# Set tick label size
fig.gca().tick_params(axis='both', which='major', labelsize=20)

ims = []
for i in range(10000):
    data = trj_ppmap_2nm[i]
    im = plt.imshow(data, aspect='auto', cmap='Greys')
    t_sim = plt.text(1.03, 1, str(i/100), fontsize=20, transform=fig.gca().transAxes, horizontalalignment='right', verticalalignment='bottom')
    ims.append([im, t_sim])
    
ani = animation.ArtistAnimation(fig, ims, blit=True, repeat=False)
ani.save('2nm_res.mp4', writer='ffmpeg', fps=50, bitrate=100000)
#plt.tight_layout()
#plt.show()